In [1]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

# PanRicci

In [2]:
import numpy as np
import networkx as nx
from panricci.utils.gfa_loader import GFALoader
from panricci.distributions.variation_graph import DistributionNodes
from panricci.ricci_flow import RicciFlow
from panricci.utils.get_source_sink import get_sources_sinks

## Load GFA

In [3]:
gfa_loader = GFALoader(undirected=False)
nodes, edges, G = gfa_loader("../data/test1.gfa")

In [4]:
nodes

{'1': {'label': 'AC', 'len': 2, 'node_depth': 1.0},
 '2': {'label': 'AA', 'len': 2, 'node_depth': 0.5},
 '3': {'label': 'CCTAAA', 'len': 6, 'node_depth': 1.0}}

In [5]:
edges

[('1', '3'), ('1', '2'), ('2', '3')]

## Probability distribution for a node

In [6]:
distribution_nodes = DistributionNodes(G, alpha=0.5)

In [7]:
distribution_nodes(node="2")

{'2': 0.5, '3': 0.5}

## Ricci Flow

In [8]:
distribution = DistributionNodes(G, alpha=0.5)
ricci_flow = RicciFlow(G, distribution,dirsave_graphs="../output/test3/ricci-flow")

In [9]:
ricci_flow(iterations=10)

Ricci-Flow: 100%|██████████| 10/10 [00:00<00:00, 795.88it/s]


## Alignment of pangenome graphs

In [10]:
path_graph1 = Path("../data/test1.gfa")
path_graph2 = Path("../data/test2.gfa")

graph1_name = path_graph1.stem
graph2_name = path_graph2.stem

# load graphs
gfa_loader = GFALoader(undirected=False)
_, _, graph1 = gfa_loader(path_graph1)
_, _, graph2 = gfa_loader(path_graph2)

dist1 = DistributionNodes(graph1, alpha=0.5)
dist2 = DistributionNodes(graph2, alpha=0.5)

# instantiate Ricci-Flow for each graph
graph1_ricci_flow = RicciFlow(graph1, dist1, dirsave_graphs=f"../output/{graph1_name}/ricci-flow")
graph2_ricci_flow = RicciFlow(graph2, dist2, dirsave_graphs=f"../output/{graph2_name}/ricci-flow")

# Run Ricci-Flow for a number of iterations
ITERATIONS=10
ricci_graph1 = graph1_ricci_flow(iterations=ITERATIONS, save_intermediate_graphs=True)
ricci_graph2 = graph2_ricci_flow(iterations=ITERATIONS, save_intermediate_graphs=True)

Ricci-Flow: 100%|██████████| 10/10 [00:00<00:00, 1023.18it/s]


2. We add source and sink nodes with arbitrary distance of $1$ to both graphs

In [11]:
ricci_graph1.edges(), ricci_graph2.edges()

(OutEdgeView([('1', '3'), ('1', '2'), ('2', '3')]),
 OutEdgeView([('1', '3'), ('1', '2'), ('2', '3')]))

In [12]:
#  graph1 
sources, sinks = get_sources_sinks(path_graph1)
ricci_graph1.add_edges_from([("source",node) for node in sources] , distance=1, label="N")
ricci_graph1.add_edges_from([(node,"sink") for node in sinks] , distance=1, label="N")
ricci_graph1.nodes(), ricci_graph1.edges()

(NodeView(('1', '2', '3', 'source', 'sink')),
 OutEdgeView([('1', '3'), ('1', '2'), ('2', '3'), ('3', 'sink'), ('source', '1')]))

In [13]:
#  graph2 
sources, sinks = get_sources_sinks(path_graph2)
ricci_graph2.add_edges_from([("source",node) for node in sources] , distance=1, label="N")
ricci_graph2.add_edges_from([(node,"sink") for node in sinks] , distance=1, label="N")
ricci_graph2.nodes(), ricci_graph2.edges()

(NodeView(('1', '2', '3', 'source', 'sink')),
 OutEdgeView([('1', '3'), ('1', '2'), ('2', '3'), ('3', 'sink'), ('source', '1')]))

3. Given source node $s$ and sink node $t$. For each graph, and each node $ u \neq s,t$ compute the vectors $$u_L = [d(s,u), d(u,t)]$$

**Graph1**

In [14]:
sp_from_source = nx.shortest_path(ricci_graph1, source="source", weight="distance", method="dijkstra")
sp_until_sink = nx.shortest_path(ricci_graph1, target="sink", weight="distance", method="dijkstra")

del sp_from_source["source"]
del sp_until_sink["sink"]
# sp_from_source
# sp_until_sink

In [15]:
costs_from_source = dict()
costs_until_sink = dict()
for start_node, path in sp_from_source.items():
    nodes = path
    edges = [(n1,n2) for n1,n2 in zip(nodes[:-1], nodes[1:])]
    cost  = np.sum([ricci_graph1.edges[e]["distance"] for e in edges])
    costs_from_source[start_node] = cost

for end_node, path in sp_until_sink.items():
    nodes = path
    edges = [(n1,n2) for n1,n2 in zip(nodes[:-1], nodes[1:])]
    cost  = np.sum([ricci_graph1.edges[e]["distance"] for e in edges])
    costs_until_sink[end_node] = cost

costs_from_source, costs_until_sink

({'1': 1,
  '3': 1.6318812829104834,
  '2': 1.3531559590013926,
  'sink': 2.6318812829104834},
 {'3': 1, '1': 1.6318812829104834, '2': 1.5, 'source': 2.6318812829104834})

In [16]:
graph1_vector_nodes = {node: np.array([costs_from_source[node], costs_until_sink[node]]) for node in ricci_graph1.nodes() if node not in ["source", "sink"]}
graph1_vector_nodes

{'1': array([1.        , 1.63188128]),
 '2': array([1.35315596, 1.5       ]),
 '3': array([1.63188128, 1.        ])}

**Graph 2**

In [17]:
sp_from_source = nx.shortest_path(ricci_graph2, source="source", weight="distance", method="dijkstra")
sp_until_sink = nx.shortest_path(ricci_graph2, target="sink", weight="distance", method="dijkstra")

del sp_from_source["source"]
del sp_until_sink["sink"]

In [18]:
costs_from_source = dict()
costs_until_sink = dict()
for start_node, path in sp_from_source.items():
    nodes = path
    edges = [(n1,n2) for n1,n2 in zip(nodes[:-1], nodes[1:])]
    cost  = np.sum([ricci_graph1.edges[e]["distance"] for e in edges])
    costs_from_source[start_node] = cost

for end_node, path in sp_until_sink.items():
    nodes = path
    edges = [(n1,n2) for n1,n2 in zip(nodes[:-1], nodes[1:])]
    cost  = np.sum([ricci_graph1.edges[e]["distance"] for e in edges])
    costs_until_sink[end_node] = cost

costs_from_source, costs_until_sink

({'1': 1,
  '3': 1.6318812829104834,
  '2': 1.3531559590013926,
  'sink': 2.6318812829104834},
 {'3': 1, '1': 1.6318812829104834, '2': 1.5, 'source': 2.6318812829104834})

In [19]:
graph2_vector_nodes = {node: np.array([costs_from_source[node], costs_until_sink[node]]) for node in ricci_graph2.nodes() if node not in ["source", "sink"]}
graph2_vector_nodes

{'1': array([1.        , 1.63188128]),
 '2': array([1.35315596, 1.5       ]),
 '3': array([1.63188128, 1.        ])}

4. Find the alignment between both graphs using [minimum_weight_full_matching](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.matching.minimum_weight_full_matching.html)

- First create a bipartite graph with nodes of both graphs in each component
- Then add the weights of matching each pair of nodes as $|| u_1 - u_2 ||$

In [20]:
import parasail

In [21]:
def compute_weight_alignment(seq1, seq2,):
    "Weight to penalize cost of aligning two nodes"
    result = parasail.sw_stats(seq1,seq2, open=10, extend=2, matrix=parasail.dnafull)
    m = result.matches
    L = max(len(seq1),len(seq2))
    w = (L-m)/L # weight based on smith-waterman
    return w

In [22]:
compute_weight_alignment("CCTAAA","CCAAAA")

0.16666666666666666

In [23]:
# remove source and sink nodes from both graphs
ricci_graph1.remove_nodes_from(["source","sink"])
ricci_graph2.remove_nodes_from(["source","sink"])

In [24]:
# create bipartite graph 
# nodes are labeled as '<node>-1', if <node> it belongs to the first graph, and '<node>-2' from the second one
# the cost/weight of an edge (u,v) correspong for the  
bipartite_graph = nx.Graph()
# nodes = []
# edges = []
for node1 in ricci_graph1.nodes():
    for node2 in ricci_graph2.nodes():
        weight_ricci_metric = np.linalg.norm(graph1_vector_nodes[node1] - graph2_vector_nodes[node2])
        weight_alignment    = compute_weight_alignment(ricci_graph1.nodes[node1]["label"], ricci_graph2.nodes[node2]["label"])
        
        weight = 0.5*weight_ricci_metric + 0.5*weight_alignment
        bipartite_graph.add_edge(node1+"-1", node2+"-2", weight=weight)

In [25]:
bipartite_graph.edges()

EdgeView([('1-1', '1-2'), ('1-1', '2-2'), ('1-1', '3-2'), ('1-2', '2-1'), ('1-2', '3-1'), ('2-2', '2-1'), ('2-2', '3-1'), ('3-2', '2-1'), ('3-2', '3-1')])

In [26]:
alignment = nx.bipartite.minimum_weight_full_matching(bipartite_graph, weight="weight")
alignment

{'2-1': '2-2',
 '3-1': '3-2',
 '1-1': '1-2',
 '2-2': '2-1',
 '3-2': '3-1',
 '1-2': '1-1'}

In [27]:
for edge in bipartite_graph.edges():
    weight = bipartite_graph.edges[edge]["weight"]
    print(edge, weight)

('1-1', '1-2') 0.25
('1-1', '2-2') 0.43848859657835293
('1-1', '3-2') 0.8468075400508581
('1-2', '2-1') 0.6884885965783529
('1-2', '3-1') 0.8634742067175248
('2-2', '2-1') 0.25
('2-2', '3-1') 0.7028867769787426
('3-2', '2-1') 0.686220110312076
('3-2', '3-1') 0.16666666666666666


In [28]:
from collections import defaultdict
optimal_alignment=defaultdict(float)
for node1, node2 in alignment.items():
    edge = (node1,node2)
    edge = tuple(sorted(edge))
    weight = bipartite_graph.edges[edge]["weight"]
    if weight <= 0.5:
        print(edge, weight)
        
        optimal_alignment[edge]=weight
        
    

('2-1', '2-2') 0.25
('3-1', '3-2') 0.16666666666666666
('1-1', '1-2') 0.25
('2-1', '2-2') 0.25
('3-1', '3-2') 0.16666666666666666
('1-1', '1-2') 0.25


In [29]:
optimal_alignment=sorted(optimal_alignment.items(), key=lambda d: (d[0],d[1]),reverse=True)

In [30]:
optimal_alignment

[(('3-1', '3-2'), 0.16666666666666666),
 (('2-1', '2-2'), 0.25),
 (('1-1', '1-2'), 0.25)]

In [31]:
import numpy as np
_costs = [ 2-2*elem[1] for elem in optimal_alignment]
panricci_metric = (np.sum(_costs))/(len(ricci_graph1)+len(ricci_graph2))
panricci_metric

0.7777777777777778

In [32]:
from panricci.panricci_similarity import PanRicciSimilarity

In [35]:
panricci = PanRicciSimilarity(alpha=0.5)
panricci_metric, alignment_ = panricci(path_graph1, path_graph2)
panricci_metric

Ricci-Flow: 100%|██████████| 10/10 [00:00<00:00, 976.49it/s]


1.0

In [34]:
alignment_

[(('3-1', '3-2'), 0.08333333333333333),
 (('2-1', '2-2'), 0.0),
 (('1-1', '1-2'), 0.0)]

In [39]:
_costs = [ 2-2*elem[1] for elem in alignment_]
(np.sum(_costs))/(len(ricci_graph1)+len(ricci_graph2))

0.9722222222222222

In [40]:
from panricci.alignment.graph_alignment import GraphAlignment

aligner = GraphAlignment()
alignment = aligner(ricci_graph1, ricci_graph2, path_graph1, path_graph2)

In [41]:
alignment

[(('3-1', '3-2'), 0.08333333333333333),
 (('2-1', '2-2'), 0.0),
 (('1-1', '1-2'), 0.0)]

In [42]:
_costs = [ 2-2*elem[1] for elem in alignment]
(np.sum(_costs))/(len(ricci_graph1)+len(ricci_graph2))

0.9722222222222222